In [214]:
import numpy as np
import scipy.signal as scipysig
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
A = np.array(
    [[-1, -4, 0, 0, 0],
     [4, -1, 0, 0, 0],
     [0, 0, -3, 1, 0],
     [0, 0, -1, -3, 0],
     [0, 0, 0, 0, -2]])
B = np.ones((5, 1))
C = np.array([1, 0, 0, 0, 0])
D = np.array([0])


dim_x = A.shape[0]
dim_u = 1
dt = 0.05
A,B,C,D,_ = scipysig.cont2discrete(system=(A,B,C,D), dt = dt)
A = np.eye(5)


In [244]:
# Simulate system
steps = 10000
std_noise = 0
U = np.zeros((dim_u, steps))
X = np.zeros((dim_x, steps + 1))
eps = 1e-2
X[:, 0] = np.random.normal(size=(dim_x))

DeltaU = np.zeros((dim_u, steps))


for i in range(steps):
    U[:, i] = eps*np.random.normal(size=(dim_u))
    X[:, i+1] = A @ X[:, i] +  np.squeeze(B * U[:, i]) #+ std_noise * np.random.normal(size=(dim_x))

Xp = X[:, 1:]
Xm = X[:, :-1]
Um = U

DeltaU =  (-Um + eps*np.random.normal(size=(Um.shape)))# - U

#print('------ CLASSICAL SYS ID  ------')
AB = Xp @ np.linalg.pinv(np.vstack((Xm, Um)))
#print(AB)
#print('------ CORRUPTED SYS ID  ------')
AB_corrupted = Xp @ np.linalg.pinv(np.vstack((Xm, Um + DeltaU)))
#print(AB_corrupted)
DeltaAB = - B @ DeltaU @ np.linalg.pinv(np.vstack((Xm, Um + DeltaU)))
#print(DeltaAB)
P = np.vstack((Xm, Um + DeltaU))
# print(np.linalg.norm(np.linalg.pinv((Um + DeltaU))))
# print(np.linalg.norm(B @ DeltaU @ np.linalg.pinv((Um + DeltaU))) )
# print(np.linalg.norm(DeltaU @ (Um + DeltaU).T))
# print(np.linalg.norm(DeltaU @ (Um + DeltaU).T))
# print(DeltaU @ (Xm).T)
P2 = np.vstack((Xm, Um))
Pinv = np.linalg.inv(P @ P.T)
Pinv2 = np.linalg.inv(P2 @ P2.T)
print(f'Norm {np.linalg.norm(Pinv)} - {np.linalg.norm(Pinv2)}')
#print(np.linalg.norm(np.hstack([Xm.T, (Um+DeltaU).T])))
L1, L2, L3, L4 = Pinv[:dim_x, :dim_x], Pinv[:dim_x, dim_x: dim_x+dim_u], Pinv[dim_x:dim_x+dim_u, :dim_x], Pinv[dim_x:dim_x+dim_u, dim_x:dim_x+dim_u]
print(DeltaAB)

# B @ DeltaU @ (Xm.T @ L1 +(Um+DeltaU).T @ L3)
# print(B)
# print(B@DeltaU @ (Xm.T @ L1 +(Um+DeltaU).T @ L3))
# print(B@DeltaU @ (Xm.T @ L2 +(Um+DeltaU).T @ L4))
# print(np.linalg.norm(L1))
# print(np.linalg.norm(L3))
# print(B @ DeltaU @ (Xm.T @ L1 +(Um+DeltaU).T @ L3))
# print(np.linalg.norm(DeltaU @ (Xm.T @ L1)))
# print(np.linalg.norm(DeltaU @ ((Um+DeltaU).T @ L3)))
# print(np.linalg.norm((Um) @((Um+DeltaU).T @ L3)))
# print(B@DeltaU @ (Xm.T @ L1 +(Um+DeltaU).T @ L3))
# #print(B@DeltaU @ (Xm.T @ L2 +(Um+DeltaU).T @ L4))
# print(B@DeltaU @ (Xm.T @ L1))
# print(B@DeltaU @ ((Um+DeltaU).T @ L3))
# print(-B@(DeltaU+Um) @ (Um+DeltaU).T @ L3)
# print(np.linalg.norm(L3))
# print(np.linalg.norm(L1))
# print(B@DeltaU @ Xm.T @ L1)
# B@DeltaU @ (DeltaU + Um).T @ L3
DA = B @ DeltaU @ (Xm.T @ L1 +(Um+DeltaU).T @ L3)
DB = B @ DeltaU @ (Xm.T @ L2 +(Um+DeltaU).T @ L4)
np.linalg.norm(DeltaAB)
print(DeltaAB)
AB_corrupted

Norm 5643238196846.64 - 5643238196846.641
[[-71416424.621 85521445.805 -104833449.142 10300869.769 64685746.231
  -0.043]
 [-87197299.508 104419104.762 -127998478.106 12577051.164 78979344.288
  -0.053]
 [-77822944.553 93193278.300 -114237694.523 11224925.094 70488480.331
  -0.047]
 [-74118603.990 88757316.092 -108800026.647 10690623.217 67133257.288
  -0.045]
 [-77884673.017 93267198.376 -114328306.841 11233828.605 70544391.164
  -0.047]]
[[-71416424.621 85521445.805 -104833449.142 10300869.769 64685746.231
  -0.043]
 [-87197299.508 104419104.762 -127998478.106 12577051.164 78979344.288
  -0.053]
 [-77822944.553 93193278.300 -114237694.523 11224925.094 70488480.331
  -0.047]
 [-74118603.990 88757316.092 -108800026.647 10690623.217 67133257.288
  -0.045]
 [-77884673.017 93267198.376 -114328306.841 11233828.605 70544391.164
  -0.047]]


array([[-71416423.617, 85521445.688, -104833448.531, 10300869.854,
        64685745.684, 0.000],
       [-87197299.059, 104419105.195, -127998478.062, 12577051.365,
        78979344.309, 0.000],
       [-77822944.523, 93193278.230, -114237693.973, 11224925.280,
        70488480.652, 0.000],
       [-74118603.852, 88757316.312, -108800026.466, 10690623.402,
        67133257.510, 0.000],
       [-77884673.551, 93267198.867, -114328306.969, 11233828.866,
        70544391.992, 0.000]])

In [243]:
B

array([[0.044],
       [0.053],
       [0.048],
       [0.045],
       [0.048]])

In [229]:
np.linalg.svd(B @ (Um) @ np.hstack([Xm.T, (Um).T]) @ Pinv2)

(array([[0.410, 0.537, -0.666, -0.254, 0.187],
        [0.501, 0.207, 0.098, 0.788, -0.276],
        [0.447, -0.524, -0.191, -0.338, -0.613],
        [0.426, 0.342, 0.714, -0.437, 0.036],
        [0.447, -0.526, 0.013, 0.104, 0.715]]),
 array([0.106, 0.001, 0.001, 0.000, 0.000]),
 array([[-0.002, -0.001, -0.016, 0.016, 0.001, 1.000],
        [0.185, 0.043, 0.619, -0.754, -0.109, 0.022],
        [0.676, 0.060, -0.134, 0.161, -0.704, -0.003],
        [0.349, 0.015, 0.656, 0.575, 0.343, 0.001],
        [0.614, 0.074, -0.412, -0.271, 0.612, -0.002],
        [0.100, -0.994, -0.002, -0.034, 0.004, -0.000]]))

In [230]:
np.linalg.svd(B @ (Um) @ np.hstack([Xm.T, (Um).T]))

(array([[-0.410, 0.375, -0.044, 0.792, -0.248],
        [-0.501, 0.075, -0.099, -0.032, 0.856],
        [-0.447, -0.333, -0.733, -0.216, -0.325],
        [-0.426, 0.575, 0.293, -0.566, -0.287],
        [-0.447, -0.642, 0.605, 0.064, -0.133]]),
 array([1077.047, 0.000, 0.000, 0.000, 0.000]),
 array([[-0.043, -0.086, -0.046, -0.045, -0.050, -0.992],
        [0.814, 0.157, 0.549, 0.042, 0.062, -0.079],
        [0.112, -0.933, 0.145, 0.019, -0.298, 0.083],
        [-0.152, -0.272, 0.222, -0.267, 0.884, -0.012],
        [-0.457, 0.153, 0.703, -0.406, -0.331, 0.009],
        [-0.303, -0.004, 0.363, 0.872, 0.118, -0.049]]))

In [231]:
np.linalg.svd(B @ (DeltaU) @ np.hstack([Xm.T, (Um + DeltaU).T]) @ Pinv2)

(array([[-0.410, 0.063, -0.760, 0.354, 0.352],
        [-0.500, -0.345, 0.587, 0.434, 0.312],
        [-0.448, 0.598, 0.194, -0.542, 0.332],
        [-0.426, 0.365, 0.042, 0.298, -0.771],
        [-0.446, -0.621, -0.193, -0.552, -0.271]]),
 array([0.108, 0.002, 0.000, 0.000, 0.000]),
 array([[-0.026, 0.042, 0.093, -0.129, -0.005, -0.986],
        [0.273, -0.004, -0.634, 0.666, -0.238, -0.153],
        [-0.089, 0.083, 0.521, 0.251, -0.806, 0.026],
        [0.756, 0.032, -0.132, -0.545, -0.334, 0.042],
        [0.570, 0.220, 0.523, 0.415, 0.426, -0.013],
        [-0.145, 0.970, -0.166, -0.089, -0.017, 0.041]]))

In [232]:
DeltaU @ (Um + DeltaU).T @ L4

array([[1.007]])

In [233]:
np.linalg.norm(DeltaU @ (Um + DeltaU).T @ L3)

0.040076263960551806

In [234]:
np.linalg.norm(DeltaU @ Xm.T @ L1)

0.09759036644973983

In [239]:
np.linalg.norm((DeltaU) @ (Um + DeltaU).T @ L4)

1.0067886176583216

In [236]:
np.linalg.norm(DeltaU @ Xm.T @ L2)

0.00018238042202081976

In [237]:
AB_corrupted

array([[18426075365.815, 1711826961.498, 10330900083.433,
        -18748363440.630, -11293231589.833, -0.000],
       [22497682023.482, 2090089067.451, 12613717271.581,
        -22891186038.511, -13788695003.639, -0.000],
       [20079014723.868, 1865389025.153, 11257649324.094,
        -20430214144.959, -12306308254.529, -0.000],
       [19123261775.051, 1776597264.948, 10721789786.822,
        -19457744246.169, -11720532977.955, -0.000],
       [20094941218.537, 1866868635.149, 11266578790.659,
        -20446419207.859, -12316069508.063, -0.000]])